# ICO Fraud detection - Downloading Data
### Objective:



In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
import plotly.offline as py
import plotly.graph_objs as go
#py.init_notebook_mode(connected=True)

## 1. Accessing Etherscam API
https://pypi.org/project/etherscan/

In [3]:
# Define if data is ready or needed to be downloaded
data_ready = False

In [4]:
#import etherscan
API_KEY = '4JPN4KUGGGRIZNRD9E3KDXCZFNF4Y9XVMP'

### 1.1. Downloading the data for the ICO using Etherscan API

In [6]:
df_contract_adress = pd.read_csv('/home/gabriel/Documents/Repos/time_series_study/data_and_models/list_icos.tsv', sep='\t')
df_contract_adress


,ico,adress,fraud
0,AMPLEFORTH,0xd46ba6d942050d489dbd938a2c909a5d5039a161,0.0
1,BAND,0xba11d00c5f74255f56a5e366f4f77f5a186d7f55,0.0
2,BANKEX,0x45245bc59219eeaaf6cd3f382e078a461ff9de7b,0.0
3,CARTESI,0x491604c0fdf08347dd1fa4ee062a822a5dd06b5d,0.0
4,CELER,0x4f9254c83eb525f9fcf346490bbb3ed28a81c667,0.0
...,...,...,...
147,SIRINLABS,0x68d57c9a1c35f63e2c83ee8e49a64e9d70528d25,1.0
148,SPARKSTER,0x971d048e737619884f2df75e31c7eb6412392328,1.0
149,STOX,0x006bea43baa3f7a6f765f14f10a1a1b08334ef45,1.0
150,WALTONCHAIN,0xb7cb1c96db6b22b0d3d9536e0108d062bd488f74,1.0


In [7]:
import requests
import json

def download_ico_data_from_etherscan(ico_name, contract_address, api_key=API_KEY):
    """Fetch data from ICOs at Etherscan using API requests.
    
    Args:
        ico_name (str): Name for the ICO.
        contract_address (str): ICO token contract adress.
        api_key (str): Etherscan API key.
    
    Returns:
        None
        
    """
    # Define starting values for variable
    block_number = 0
    response_len = 10000
    status_code = 200
    response_list = []
    counter = 1 
    
    # Makes requests while reponses are successful or len < 10000
    while response_len == 10000 and status_code == 200:
        response = requests.get(f'http://api.etherscan.io/api?module=account&action=txlist&address={contract_address}&startblock={block_number}&endblock=99999999&sort=asc&apikey={api_key}')
        status_code = response.__dict__.get('status_code')
        response_len = len(response.json().get('result'))
        block_number = response.json().get('result')[-1].get('blockNumber')
        
        print(f'{ico_name} iteration number: {str(counter)}, response length: {str(response_len)}, status code: {str(status_code)}')
        counter += 1
        
        # Check status_code
        if status_code == 200:
            response_list.extend(response.json().get('result'))
            
    pd.DataFrame(response_list).to_csv(f'../data_and_models/api_etherscan/{ico_name}.csv',index=False)
       
            

In [10]:
df_contract_adress[df_contract_adress.fraud == 0].shape

(65, 3)

In [11]:
%%time
if not data_ready:
    for index, row in df_contract_adress.iterrows():
        if row.fraud == 1:
            download_ico_data_from_etherscan(row.ico, row.adress)

ARBITRAGE iteration number: 1, response length: 10000, status code: 200
ARBITRAGE iteration number: 2, response length: 10000, status code: 200
ARBITRAGE iteration number: 3, response length: 10000, status code: 200
ARBITRAGE iteration number: 4, response length: 10000, status code: 200
ARBITRAGE iteration number: 5, response length: 7819, status code: 200
BILLIONAIRE iteration number: 1, response length: 3503, status code: 200
BITCOIN CROWN iteration number: 1, response length: 2735, status code: 200
BITCOIN MAX iteration number: 1, response length: 1680, status code: 200
BLOCKMALL iteration number: 1, response length: 2068, status code: 200
CARNOMIC iteration number: 1, response length: 4135, status code: 200
CHANGENOW iteration number: 1, response length: 10000, status code: 200
CHANGENOW iteration number: 2, response length: 10000, status code: 200
CHANGENOW iteration number: 3, response length: 10000, status code: 200
CHANGENOW iteration number: 4, response length: 10000, status c

## 1.2 Using BigQuery API to fetch data

```python
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client()

query = f"""
    SELECT * FROM `bigquery-public-data.crypto_ethereum.token_transfers`
    WHERE TOKEN_ADDRESS = '{token_adress}'
"""
query_job = client.query(query)  # Make an API request.

print("The query data:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print("name={}, count={}".format(row[0], row["total_people"]))
```